In [34]:
 @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='55879393-3d44-426c-ac44-d401f24e63b1', project_access_token='p-15d21cd01b2182de4b20021c385dddbe92ffb6ef')
pc = project.project_context


## Scraping Data from a Table on a Website into a Pandas DataFrame


#### Scapping the data from the Wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)


## Inspect HTML code of website
Based on the HTML codes, each row of the data are stored in `<tr>..</tr>`. This is the row information. Each row has a corresponding `<td>..</td>` or cell data information.

## Import Libraries

In [1]:
import requests # for getting the HTML contents of the website
import lxml.html as lh # for parsing the relevant fields
import pandas as pd # for converting to a Pandas DataFrame and storing the data
import numpy as np

## Send the GET request to get the content of the website

In [2]:
#sent the GET Request 
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
doc

<Element html at 0x7f84b5db1c28>

## Parse data that are stored between `<tr>..</tr>` 

In [3]:
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
tr_elements #Each element in tr_element contrains the rows of the table. since each row has three columns, the length of each row should be three
#Check the length of the first 12 rows
ncolumns=[len(T) for T in tr_elements[0:10]]
ncolumns

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

## EXtract the Elements in tr_elements as a list of list or a dictionary

#### Parse Table Header


In [4]:
col=[]
i=0
#note tr_elements[0] is the header row. It has three contents representing the columns
# Extract the content into a list 
for t in tr_elements[0]:
    i=i+1
    name=t.text_content()
    name = name.rstrip("\n") #just to stripe the trailling "\n"
    col.append((name,[]))
col

[('Postal Code', []), ('Borough', []), ('Neighborhood', [])]

#### Parse the row Elements and append them to the corresponding tuple in the list col 

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        data = data.rstrip("\n") #just to stripe the trailling "\n"
  
        #Append the data to the i'th column in the list col 
        col[i][1].append(data)
        #Increment i for the next column
        i=i+1

In [6]:
[len(C) for (title,C) in col] # check the length of each coloumn


[181, 181, 181]

## Create a Pandas DataFrame

In [7]:
Dict={title:column for (title,column) in col} #create a python dictionary with the data in col 
df=pd.DataFrame(Dict) # convert dictionary to a Pandas DataFrame

In [8]:
df['Postal Code'].replace('', np.nan, inplace=True)
df
df.dropna(inplace=True)

In [9]:
df.tail()

,Postal Code,Borough,Neighborhood
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,


## Preprossessing

In [14]:
# delete rows with unassigned Borough
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(inplace=True)

#check to make sure there are no duplicate Postal Codes
print('the dataset has ', len(df['Postal Code']), 'postal Codes')
print('the dataset has ', len(df['Postal Code'].unique()), 'unique postal Codes')

#reset the index 
df.reset_index(drop=True, inplace=True)
df.head()

the dataset has  103 postal Codes
the dataset has  103 unique postal Codes


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [36]:
@hidden_cell

project.save_data(data=df.to_csv(index=False),file_name='df.csv',overwrite=True)

{'file_name': 'df.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneprojectweek3-donotdelete-pr-u7gb3mhhsnrn0a',
 'asset_id': 'b1b5541b-1c1c-43e2-8ab7-bbcece4d166e'}

In [37]:
print(df.shape)

(103, 3)
